# Script for one day

In [1]:
import requests
import json
import pandas as pd
from datetime import date
import openpyxl
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time
load_dotenv()

True

In [2]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [3]:

# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

In [4]:

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text }")

Data retrieved successfully


In [5]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)


# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [11]:
#unique_change_times = df['changeTime'].unique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [6]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# List to store all campaign data
all_campaign_data_guten = []
all_campaign_data_giper = []
all_campaign_data_kitchen = []


# Iterate over each chunk and send a POST request
for idx, (chunk_guten, chunk_giper, chunk_kitchen) in enumerate(zip(campaign_chunks_guten, campaign_chunks_giper, campaign_chunks_kitchen)):
    
    # Send the POST request
    response_guten = requests.post(url, params=query_params, json=chunk_guten, headers=headers_guten)
    time.sleep(1)
    response_giper = requests.post(url, params=query_params, json=chunk_giper, headers=headers_giper)
    time.sleep(1)
    response_kitchen = requests.post(url, params=query_params, json=chunk_kitchen, headers=headers_kitchen)
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
        # Parse the JSON response
        data_guten = response_guten.json()
        data_giper = response_giper.json()
        data_kitchen = response_kitchen.json()
        all_campaign_data_guten.extend(data_guten)
        all_campaign_data_giper.extend(data_giper)
        all_campaign_data_kitchen.extend(data_kitchen)
        print("Data retrieved successfully")
        #print(f"Response for Chunk {idx + 1}: {data_guten}")
    else:
        print(f"Error for Chunk {idx + 1}: {response_guten.status_code,response_giper.status_code,response_kitchen.status_code}, {response_guten.text,response_giper.text,response_kitchen.text}")
        
        
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully
Combined Campaign Data


In [7]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-07,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-07,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
86,2024-07-23,2024-06-05,2024-06-05,Braun Триммеры,17748698,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
87,2024-07-23,2024-06-05,2024-06-05,Braun Эпиляторы,17741906,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
88,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,21008536,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries
89,2100-01-01,2024-10-30,2025-01-09,Миксер планет_30.10.2024,21008249,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [12]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Automatically get yesterday's date
yesterday = date.today().replace(day=date.today().day - 1)

specific_date = str(yesterday)  # Replace with your desired date

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [13]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_guten, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_guten.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
#with open("response_data_guten.txt", "w", encoding="utf-8") as file:
  #json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)


Data retrived successfully


### Giper

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id,"dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_giper, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_giper.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

Data retrived successfully
Combined Campaign Data


### Kitchen

In [ ]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "dates": [specific_date]} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_kitchen, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_kitchen.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

Data retrived successfully
Combined Campaign Data


In [16]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,dates,days,advertId
0,4249,113,2.66,16.48,1861.74,14,3,2.65,3,10611,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",21554091
1,4177,112,2.68,3.27,366.65,9,3,2.68,3,1887,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",21497699
2,4974,235,4.72,1.53,359.45,13,3,1.28,3,15505,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",19297685
3,2877,71,2.47,6.61,469.33,3,2,2.82,2,21818,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",18870595
4,3661,134,3.66,15.76,2111.53,8,1,0.75,1,3213,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",21883272
5,7441,202,2.71,5.26,1062.27,24,1,0.50,1,1705,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",23231292
6,8398,182,2.17,5.67,1031.44,18,6,3.30,6,7582,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",23230882
7,3014,108,3.58,17.38,1876.87,7,1,0.93,1,3340,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",21883285
8,3856,146,3.79,4.21,614.87,18,6,4.11,6,7811,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",22846446
9,1667,22,1.32,9.47,208.37,1,0,0.00,0,0,[2025-03-06],"[{'date': '2025-03-06T03:00:00+03:00', 'views'...",18870853


In [17]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-06 03:00:00,171562371,Ингалятор небулайзер компрессорный CN-235,364,8,2.20,21.10,168.76,0,0,0.00,0,0,21554091
1,2025-03-06 03:00:00,171562371,Ингалятор небулайзер компрессорный CN-235,2670,67,2.51,17.22,1153.56,9,1,1.49,1,3537,21554091
2,2025-03-06 03:00:00,171562371,Ингалятор небулайзер компрессорный CN-235,1215,38,3.13,14.20,539.42,5,2,5.26,2,7074,21554091
3,2025-03-06 03:00:00,242524788,"Гель для бритья Series очищающий с углем, 200 мл",96,1,1.04,8.49,8.49,0,0,0.00,0,0,21497699
4,2025-03-06 03:00:00,209695451,Гель для бритья Series Moisturizing 200 мл,91,0,0.00,0.00,7.55,0,0,0.00,0,0,21497699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,2025-03-06 03:00:00,235394721,Точилка для ножей электрическая CX-110,21,0,0.00,0.00,3.14,0,0,0.00,0,0,20124734
558,2025-03-06 03:00:00,208811603,Точилка для ножей электрическая CC-120,32,1,3.13,5.98,5.98,0,0,0.00,0,0,20124734
559,2025-03-06 03:00:00,327350559,,68,0,0.00,0.00,11.29,0,0,0.00,0,0,23408417
560,2025-03-06 03:00:00,327350559,,339,5,1.47,11.14,55.72,2,0,0.00,0,0,23408417


In [18]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,23806943,2025-03-06 03:00:00,Вакууматор для продуктов с запайкой VC 11,612,22,3.400000,8.986667,97.56,1,0,0.0,0,0,18870595,WB-GutenTech,Wildberries
1,25623718,2025-03-06 03:00:00,Бритва мужская электрическая S3 300BT с триммером,41,3,9.636667,1.406667,4.22,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
2,25624340,2025-03-06 03:00:00,Электробритва Series 3 300s,98,9,4.763333,0.183333,8.13,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
3,25628178,2025-03-06 03:00:00,Бритва мужская электрическая S3 300s,145,5,1.753333,0.503333,11.54,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
4,25725047,2025-03-06 03:00:00,Триммер для бороды и усов BT3222,1,0,0.000000,0.000000,0.06,0,0,0.0,0,0,21212396,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,70008252,2025-03-06 03:00:00,Тостер для хлеба с поддоном для крошек Artisan...,238,1,0.240000,8.470000,44.14,0,0,0.0,0,0,21006669,WB-KitchenAid,Wildberries
361,243235996,2025-03-06 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",82,1,0.756667,1.860000,9.87,0,0,0.0,0,0,21008536,WB-KitchenAid,Wildberries
362,243235999,2025-03-06 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",170,8,3.553333,1.540000,20.82,0,0,0.0,0,0,21008536,WB-KitchenAid,Wildberries
363,245313750,2025-03-06 03:00:00,Тостер на 4 ломтика 5KMT4109E,213,5,2.526667,6.473333,33.44,2,0,0.0,0,0,21006669,WB-KitchenAid,Wildberries


In [19]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,23806943,2025-03-06 03:00:00,Вакууматор для продуктов с запайкой VC 11,612,22,97.56,1,0,0,0,18870595,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания
1,25623718,2025-03-06 03:00:00,Бритва мужская электрическая S3 300BT с триммером,41,3,4.22,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
2,25624340,2025-03-06 03:00:00,Электробритва Series 3 300s,98,9,8.13,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25628178,2025-03-06 03:00:00,Бритва мужская электрическая S3 300s,145,5,11.54,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
4,25725047,2025-03-06 03:00:00,Триммер для бороды и усов BT3222,1,0,0.06,0,0,0,0,21212396,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Триммеры_07.11.2024,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,70008252,2025-03-06 03:00:00,Тостер для хлеба с поддоном для крошек Artisan...,238,1,44.14,0,0,0,0,21006669,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Тостеры_30.10.2024,Идут показы,Автоматическая кампания
361,243235996,2025-03-06 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",82,1,9.87,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
362,243235999,2025-03-06 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",170,8,20.82,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
363,245313750,2025-03-06 03:00:00,Тостер на 4 ломтика 5KMT4109E,213,5,33.44,2,0,0,0,21006669,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Тостеры_30.10.2024,Идут показы,Автоматическая кампания


## Insert the data

In [20]:
password = os.getenv('ClickHouse')

In [21]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [22]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
